In [1]:
import face_recognition
import cv2
import pandas as pd
import glob
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage import color
from time import time

In [2]:
# Read images, calculate face encodings and save encodings to csv
pictures = glob.glob("D:/dataset2/train/*")
known_face_names = []
for i in range(0, len(pictures)):
    e = pictures[i].split('\\')[1].split('.')[0]
    known_face_names = np.append(known_face_names,e)
known_face_encodings = []
for i in range(0, len(pictures)):
    a = face_recognition.load_image_file(pictures[i])
    known_face_encodings = known_face_encodings + [face_recognition.face_encodings(a)[0]]
    
pd.DataFrame(known_face_names).to_csv("D:/New folder/names.csv", index=False)
pd.DataFrame(known_face_encodings).to_csv("D:/New folder/encodings.csv", index=False)

In [6]:
# reading single image for benchmarking
start_full = time()
known_face_encodings = pd.read_csv("D:/New folder/encodings.csv")
known_face_names = pd.read_csv("D:/New folder/names.csv")
# known_face_encodings = pd.DataFrame(known_face_encodings)
# known_face_names = pd.DataFrame(known_face_names)
face_locations = []
face_encodings = []
face_names = []
each = []
rgb_small_frame = face_recognition.load_image_file("D:/dataset2/Barack.jpg")
# pic = np.array(mpimg.imread("D:/dataset2/Zuck.jpg"))
# print(rgb_small_frame)
# plt.imshow(pic)
# print(pic.shape)
# rgb_small_frame = cv2.resize(pic, (0, 0), fx=0.25, fy=0.25)
# plt.imshow(rgb_small_frame)
for i in range(100):
    start_each = time()
    face_locations = face_recognition.face_locations(rgb_small_frame) # model = 'cnn'
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations) #  increase num_jitters
#     print(face_locations)
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"
        if True in matches:
            dist = face_recognition.api.face_distance(known_face_encodings, face_encoding)
#             print(dist)
            closest_match_index = np.argmin(dist)
            name = list(known_face_names.iloc[closest_match_index])[0].split('_')[0]
        face_names.append(name)
#     cv2.imshow('Picture', pic)
#     cv2.waitKey(0)
    each.append(time()-start_each)
cv2.destroyAllWindows()
print(time()-start_full)

231.59244465827942


In [7]:
print(face_names.count('Barack'))
print(sum(each)/len(each))

100
2.314477889537811


In [ ]:
# Reading video stream
video_capture = cv2.VideoCapture(0)
known_face_encodings = pd.read_csv("D:/New folder/encodings.csv")
known_face_names = pd.read_csv("D:/New folder/names.csv")
# known_face_encodings = pd.DataFrame(known_face_encodings)
# known_face_names = pd.DataFrame(known_face_names)
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    if process_this_frame:
        face_locations = face_recognition.face_locations(rgb_small_frame) # model = 'cnn'
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations) #  increase num_iters
        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            if True in matches:
                dist = face_recognition.api.face_distance(known_face_encodings, face_encoding)
                closest_match_index = np.argmin(dist)
                name = list(known_face_names.iloc[closest_match_index])[0]
                face_names.append(name)
    process_this_frame = not process_this_frame
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
video_capture.release()
cv2.destroyAllWindows()
